In [ ]:
# ============================================================
# STEP 1: LOAD DATA
# ============================================================
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
a
print("📥 Loading data...")

# Local paths (change to Kaggle paths for submission)
X_train_raw = pd.read_csv('X_train.csv')
y_train = pd.read_csv('y_train.csv')
X_test_raw = pd.read_csv('X_test.csv')

# Sample for fast local testing (remove for full run)
SAMPLE = 1000
np.random.seed(42)
idx = np.random.choice(len(X_train_raw), SAMPLE, replace=False)
X_train_raw = X_train_raw.iloc[idx].reset_index(drop=True)

y_train = y_train.iloc[idx].reset_index(drop=True)
print(f"Target zeros: {(y_train['MathScore']==0).mean()*100:.1f}%")

X_test_raw = X_test_raw.iloc[:SAMPLE].reset_index(drop=True)
print(f"Train: {X_train_raw.shape}, Test: {X_test_raw.shape}")



# Save test IDs (use Unnamed: 0, not sequential)
test_ids = X_test_raw['Unnamed: 0'].values

In [ ]:
# ============================================================
# STEP 2: REMOVE LEAKY COLUMNS + CLEAN DATA
# ============================================================
print("🧹 Removing leaky columns...")

# These columns are 100% NaN in test = data leakage
leaky_patterns = ['math_q', '_average_score', '_total_timing']
leaky_cols = [c for c in X_train_raw.columns if any(p in c.lower() for p in leaky_patterns)]
print(f"   Removing {len(leaky_cols)} leaky columns")

# Clean train
X_train_full = X_train_raw.drop(columns=['Unnamed: 0'] + leaky_cols, errors='ignore')
X_train_full = X_train_full.select_dtypes(include=[np.number]).astype(np.float32)

# Clean test
X_test_full = X_test_raw.drop(columns=['Unnamed: 0'] + leaky_cols, errors='ignore')
X_test_full = X_test_full.select_dtypes(include=[np.number]).astype(np.float32)

print(f"   Final features: {X_train_full.shape[1]}")

In [ ]:
# ============================================================
# STEP 3: GPU-ACCELERATED ENSEMBLE WITH TARGET NORMALIZATION
# ============================================================
import catboost as cb
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import roc_auc_score
import lightgbm as lgb
import xgboost as xgb
import gc

print("🚀 GPU-Accelerated Ensemble with Target Normalization...")

y = y_train['MathScore'].values.ravel()
is_zero = (y == 0).astype(int)

# ============ STAGE 1: CATBOOST CLASSIFIER (GPU) ============
print("\n📊 Stage 1: CatBoost Classifier (GPU)...")
skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
clf_oof = np.zeros(len(y))
clf_test = np.zeros(len(X_test_full))

for fold, (tr_idx, val_idx) in enumerate(skf.split(X_train_full, is_zero)):
    clf = cb.CatBoostClassifier(iterations=100, depth=5, learning_rate=0.2,
                                 random_seed=42+fold, verbose=0,
                                 early_stopping_rounds=20)
    clf.fit(X_train_full.iloc[tr_idx], is_zero[tr_idx],
            eval_set=(X_train_full.iloc[val_idx], is_zero[val_idx]))
    clf_oof[val_idx] = clf.predict_proba(X_train_full.iloc[val_idx])[:, 1]
    clf_test += clf.predict_proba(X_test_full)[:, 1] / 3
    del clf
    gc.collect()

print(f"   Classifier AUC: {roc_auc_score(is_zero, clf_oof):.4f}")

# ============ STAGE 2: GPU REGRESSORS WITH TARGET NORMALIZATION ============
print("\n🔬 Stage 2: GPU Regressors with Target Normalization...")

nz_mask = y > 0
X_nz = X_train_full[nz_mask].values.astype(np.float32)
y_nz = y[nz_mask]
X_test_arr = X_test_full.values.astype(np.float32)

# Target normalization
y_mean = y_nz.mean()
y_std = y_nz.std()
y_nz_norm = (y_nz - y_mean) / y_std
print(f"   Target stats: mean={y_mean:.2f}, std={y_std:.2f}")

kf = KFold(n_splits=3, shuffle=True, random_state=42)

# --- MODEL 1: CatBoost Regressor (GPU) ---
print("\n   🔥 Training CatBoost Regressor (GPU)...")
cb_oof = np.zeros(len(y_nz))
cb_test = np.zeros(len(X_test_arr))

for fold, (tr_idx, val_idx) in enumerate(kf.split(X_nz)):
    cb_reg = cb.CatBoostRegressor(
        iterations=1500, depth=8, learning_rate=0.05,
        l2_leaf_reg=5, random_seed=42+fold,
        verbose=0, early_stopping_rounds=150
    )
    cb_reg.fit(X_nz[tr_idx], y_nz_norm[tr_idx], eval_set=(X_nz[val_idx], y_nz_norm[val_idx]))
    cb_oof[val_idx] = cb_reg.predict(X_nz[val_idx])
    cb_test += cb_reg.predict(X_test_arr) / 3
    del cb_reg
    gc.collect()

cb_oof_denorm = cb_oof * y_std + y_mean
cb_test_denorm = cb_test * y_std + y_mean
cb_r2 = 1 - (np.sum((y_nz - cb_oof_denorm)**2) / np.sum((y_nz - y_nz.mean())**2))
print(f"      CatBoost R²: {cb_r2:.4f}")

# --- MODEL 2: LightGBM (GPU) ---
print("\n   🔥 Training LightGBM (GPU)...")
lgb_oof = np.zeros(len(y_nz))
lgb_test = np.zeros(len(X_test_arr))

X_nz_clean = np.nan_to_num(X_nz, nan=0.0, posinf=0.0, neginf=0.0)
X_test_clean = np.nan_to_num(X_test_arr, nan=0.0, posinf=0.0, neginf=0.0)

for fold, (tr_idx, val_idx) in enumerate(kf.split(X_nz)):
    lgb_reg = lgb.LGBMRegressor(
        n_estimators=1500, learning_rate=0.05, max_depth=8,
        num_leaves=128, reg_alpha=0.1, reg_lambda=1.0,
        min_child_samples=20, subsample=0.8, colsample_bytree=0.8,
        n_jobs=-1,
        random_state=42+fold, verbose=-1
    )
    lgb_reg.fit(X_nz_clean[tr_idx], y_nz_norm[tr_idx],
                eval_set=[(X_nz_clean[val_idx], y_nz_norm[val_idx])],
                callbacks=[lgb.early_stopping(150, verbose=False)])
    lgb_oof[val_idx] = lgb_reg.predict(X_nz_clean[val_idx])
    lgb_test += lgb_reg.predict(X_test_clean) / 3
    del lgb_reg
    gc.collect()

lgb_oof_denorm = lgb_oof * y_std + y_mean
lgb_test_denorm = lgb_test * y_std + y_mean
lgb_r2 = 1 - (np.sum((y_nz - lgb_oof_denorm)**2) / np.sum((y_nz - y_nz.mean())**2))
print(f"      LightGBM R²: {lgb_r2:.4f}")

# --- MODEL 3: XGBoost (GPU) ---
print("\n   🔥 Training XGBoost (GPU)...")
xgb_oof = np.zeros(len(y_nz))
xgb_test = np.zeros(len(X_test_arr))

for fold, (tr_idx, val_idx) in enumerate(kf.split(X_nz)):
    xgb_reg = xgb.XGBRegressor(
        n_estimators=1500, learning_rate=0.05, max_depth=8,
        reg_alpha=0.1, reg_lambda=1.0, subsample=0.8, colsample_bytree=0.8,
        n_jobs=-1,
        random_state=42+fold, verbosity=0,
        early_stopping_rounds=150
    )
    xgb_reg.fit(X_nz_clean[tr_idx], y_nz_norm[tr_idx],
                eval_set=[(X_nz_clean[val_idx], y_nz_norm[val_idx])], verbose=False)
    xgb_oof[val_idx] = xgb_reg.predict(X_nz_clean[val_idx])
    xgb_test += xgb_reg.predict(X_test_clean) / 3
    del xgb_reg
    gc.collect()

xgb_oof_denorm = xgb_oof * y_std + y_mean
xgb_test_denorm = xgb_test * y_std + y_mean
xgb_r2 = 1 - (np.sum((y_nz - xgb_oof_denorm)**2) / np.sum((y_nz - y_nz.mean())**2))
print(f"      XGBoost R²: {xgb_r2:.4f}")

# ============ BLEND GPU MODELS ============
print("\n📊 GPU Model Summary:")
print(f"   CatBoost:  R²={cb_r2:.4f}")
print(f"   LightGBM:  R²={lgb_r2:.4f}")
print(f"   XGBoost:   R²={xgb_r2:.4f}")

total_r2 = cb_r2 + lgb_r2 + xgb_r2
w_cb = cb_r2 / total_r2
w_lgb = lgb_r2 / total_r2
w_xgb = xgb_r2 / total_r2
print(f"\n🔗 Weights: CB={w_cb:.3f}, LGB={w_lgb:.3f}, XGB={w_xgb:.3f}")

blend_oof_nz = w_cb * cb_oof_denorm + w_lgb * lgb_oof_denorm + w_xgb * xgb_oof_denorm
blend_test = w_cb * cb_test_denorm + w_lgb * lgb_test_denorm + w_xgb * xgb_test_denorm

blend_r2 = 1 - (np.sum((y_nz - blend_oof_nz)**2) / np.sum((y_nz - y_nz.mean())**2))
print(f"   Blend R² (non-zero): {blend_r2:.4f}")

# Map back to full arrays
reg_oof = np.zeros(len(y))
reg_oof[nz_mask] = blend_oof_nz

# ============ COMBINE WITH CLASSIFIER ============
final_oof = reg_oof.copy()
final_oof[clf_oof > 0.5] = 0

final_test = blend_test.copy()
final_test[clf_test > 0.5] = 0

oof_r2 = 1 - (np.sum((y - final_oof)**2) / np.sum((y - y.mean())**2))
print(f"\n🎯 FINAL OOF R² = {oof_r2:.4f}")

In [ ]:
# ============================================================
# STEP 4: GENERATE SUBMISSION
# ============================================================
print("📤 Generating submission...")

submission = pd.DataFrame({
    'ID': test_ids,
    'MathScore': final_test
})

submission['MathScore'] = submission['MathScore'].clip(lower=0)
submission.to_csv('submission.csv', index=False)

print(f"✅ Saved submission.csv")
print(f"   Shape: {submission.shape}")
print(f"   Zeros: {(submission['MathScore'] == 0).sum()} ({(submission['MathScore'] == 0).mean()*100:.1f}%)")
print(f"   Mean: {submission['MathScore'].mean():.2f}")
print(f"\n🏆 OOF R² = {oof_r2:.4f}")